In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import L1
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv("data/processed/traindata.csv")
test = pd.read_csv("data/processed/testdata.csv")

In [3]:
train.head()

,Unnamed: 0,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,...,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH,service_icmp
0,0,0,239,486,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,1,0,235,1337,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,2,0,219,1337,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,3,0,217,2032,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,4,0,217,2032,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [4]:
train.shape

(492998, 121)

In [5]:
test.head()

,Unnamed: 0,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,...,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH,service_pm_dump,service_red_i,service_urh_i
0,0,0,105,146,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1,0,105,146,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,5,0,29,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,6,0,105,146,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,8,0,223,185,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0


In [6]:
test.shape

(292299, 121)

In [7]:
encoder = LabelEncoder()
train['labels'] = encoder.fit_transform(train['labels'])
test['labels'] = encoder.transform(test['labels'])

In [8]:
scale = StandardScaler()

train_x = scale.fit_transform(train.drop("labels", axis=1).values)
train_y = train.labels.to_numpy()
test_x = scale.transform(test.drop("labels", axis=1).values)
test_y = test.labels.to_numpy()
n_features = train_x.shape[1]

In [9]:
test_x, cv_x, test_y, cv_y = train_test_split(test_x, test_y, test_size=0.5, random_state=11)

In [13]:
reg = 0.0525

model = Sequential()
model.add(Dense(120, activation='relu', kernel_initializer='he_normal', kernel_regularizer=L1(reg), input_shape=(n_features,)))
model.add(Dense(100, activation='relu', kernel_initializer='he_normal', kernel_regularizer=L1(reg)))
model.add(Dense(80, activation='relu', kernel_initializer='he_normal', kernel_regularizer=L1(reg)))
model.add(Dense(60, activation='relu', kernel_initializer='he_normal', kernel_regularizer=L1(reg)))
model.add(Dense(40, activation='relu', kernel_initializer='he_normal', kernel_regularizer=L1(reg)))
model.add(Dense(21, activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0011),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [14]:
model.fit(train_x, train_y, epochs=10, batch_size=128)

Epoch 1/10
   7/3852 [..............................] - ETA: 34s - loss: 271.3984 - accuracy: 0.2634      

2022-08-30 13:43:17.876227: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


3852/3852 [==============================] - 28s 7ms/step - loss: 6.8156 - accuracy: 0.8786
Epoch 2/10
3852/3852 [==============================] - 28s 7ms/step - loss: 0.7851 - accuracy: 0.9815
Epoch 3/10
3852/3852 [==============================] - 27s 7ms/step - loss: 0.7575 - accuracy: 0.9815
Epoch 4/10
3852/3852 [==============================] - 27s 7ms/step - loss: 0.7394 - accuracy: 0.9815
Epoch 5/10
3852/3852 [==============================] - 28s 7ms/step - loss: 0.7263 - accuracy: 0.9815
Epoch 6/10
3852/3852 [==============================] - 27s 7ms/step - loss: 0.7157 - accuracy: 0.9815
Epoch 7/10
3852/3852 [==============================] - 27s 7ms/step - loss: 0.7077 - accuracy: 0.9815
Epoch 8/10
3852/3852 [==============================] - 28s 7ms/step - loss: 0.7008 - accuracy: 0.9815
Epoch 9/10
3852/3852 [==============================] - 28s 7ms/step - loss: 0.6945 - accuracy: 0.9815
Epoch 10/10
3852/3852 [==============================] - 28s 7ms/step - loss: 0.6900

In [15]:
model.evaluate(train_x, train_y)

   22/15407 [..............................] - ETA: 1:18 - loss: 0.6173 - accuracy: 1.0000

2022-08-30 13:48:48.291388: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


15407/15407 [==============================] - 69s 4ms/step - loss: 0.6804 - accuracy: 0.9821


[0.6804103851318359, 0.9821439385414124]

In [16]:
model.evaluate(cv_x, cv_y)

4568/4568 [==============================] - 21s 5ms/step - loss: 1.2198 - accuracy: 0.8505


[1.2197932004928589, 0.8505018949508667]

In [17]:
model.evaluate(test_x, test_y)

4568/4568 [==============================] - 21s 5ms/step - loss: 1.2163 - accuracy: 0.8510


[1.2163195610046387, 0.8510023951530457]